In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
# import hvplot.pandas
# import holoviews as hv
import panel as pn
import pandas as pd
import numpy as np
import param as pm
import random
import math
import plotly.express as px
pn.extension('plotly', sizing_mode="stretch_width")
# hv.extension("bokeh")
# import os, sys
# sys.path.append('/home/bk/LTF-base/aaa-tokenomics')
# pn.extension('plotly')
import warnings
warnings.filterwarnings('ignore')

In [3]:
from AAA.sim_handler import SimHandler
from AAA.config import sim_conf

In [4]:
class Input(pm.Parameterized):

    days = pm.Integer(default=60, bounds=(10, 365))
    user_count = pm.Integer(default=10, bounds=(10, 100))
    #------------------------------------Bond Terms
    bond_control_variable = pm.Number(default = 0, bounds=(0,1))
    vesting_term = pm.Integer(default = 5, bounds=(0,100))
    min_price = pm.Number(default = 0.8, bounds=(0,10), step = 0.1)
    max_payout = pm.Integer(default = 10000, bounds=(0,100000))
    fee = pm.Number(default = 100, bounds=(0,100))
    max_debt = pm.Number(default = 0.6, bounds=(0,10), step = 0.01)
    #------------------------------------Adjustments
    add = pm.Boolean(default = True)
    rate = pm.Number(default = 0.02, bounds = (0,1), step=0.01)
    target = pm.Number(default = 0.2, bounds = (0,5), step = 0.01)
    buffer = pm.Integer(default = 2, bounds = (0,10))
    lastBlock = pm.Number(default = 0, bounds = None)
    
    # create a button that when pushed triggers 'run_sim'
    run_sim = pm.Action(
        lambda self: self.param.trigger('run_sim'),
        label='Run Simulation')
    
    @pm.depends('run_sim')
    def view(self):
        ##modify sim configs
        sim_conf['days'] = self.days
        sim_conf['user_count'] = self.user_count
        #-----------
        sim_conf['bond_config']['bond_term']['bond_control_variable'] = self.bond_control_variable
        sim_conf['bond_config']['bond_term']['vesting_term'] = self.vesting_term
        sim_conf['bond_config']['bond_term']['min_price'] = self.min_price
        sim_conf['bond_config']['bond_term']['max_payout'] = self.max_payout
        sim_conf['bond_config']['bond_term']['fee'] = self.fee
        sim_conf['bond_config']['bond_term']['max_debt'] = self.max_debt
        #--------
        sim_conf['bond_config']['adjustment']['add'] = self.add
        sim_conf['bond_config']['adjustment']['rate'] = self.rate
        sim_conf['bond_config']['adjustment']['target'] = self.target
        sim_conf['bond_config']['adjustment']['buffer'] = self.buffer
        sim_conf['bond_config']['adjustment']['lastBlock'] = self.lastBlock
  
        print(sim_conf)
        s = SimHandler(sim_conf)
        df, dfa, f = s.run()
        p1 = pn.pane.Plotly(f[0])
        p2 = pn.pane.Plotly(f[1], config={'responsive': True})
        p3 = pn.pane.Plotly(f[2], config={'responsive': True})
        p4 = pn.pane.Plotly(f[3], config={'responsive': True})

        view = pn.Column(
            p1,
            p2,
            p3,
            p4,            
#             f[0],
#             f[1],
#             f[2],
#             f[3],
            df,
            dfa,
        )
        return view
    
    @pm.depends('run_sim')
    def view_model(self):
        p1 = pn.Row(
            pn.Column(
                '### Sim Config',
                self.param.days,
                self.param.user_count,
                pn.Param(
                self.param.run_sim, widgets={
                    'run_sim': {
                        'button_type': 'success'}})
            ),
            pn.Column(
                '### Bond terms',
                self.param.bond_control_variable,
                self.param.vesting_term,
                self.param.min_price,
                self.param.max_payout,
                self.param.fee,
                self.param.max_debt),
            pn.Column(
                '### Adjustments',
                self.param.add,
                self.param.rate,
                self.param.target,
                self.param.buffer,
                self.param.lastBlock)
        )
        return pn.Column(p1, self.view())

In [5]:
h = Input()

In [6]:
h.view_model()

{'days': 60, 'user_count': 10, 'bond_count': 1, 'user_config': {'balances': {'DAI': 10000, 'AHM': 0, 'sAHM': 0, 'ETH': 0}}, 'treasury_config': {'staked_native_assets': {}}, 'bond_config': {'bond_term': {'bond_control_variable': 0, 'vesting_term': 5, 'min_price': 0.8, 'max_payout': 10000, 'fee': 100, 'max_debt': 0.6}, 'principle': 'DAI', 'is_Liquidity_Bond': False, 'adjustment': {'add': True, 'rate': 0.02, 'target': 0.2, 'buffer': 2, 'lastBlock': 0}}}


Column(sizing_mode='stretch_width')
    [0] Row(sizing_mode='stretch_width')
        [0] Column(sizing_mode='stretch_width')
            [0] Markdown(str, sizing_mode='stretch_width')
            [1] IntSlider(end=365, name='Days', sizing_mode='stretch_width', start=10, value=60)
            [2] IntSlider(end=100, name='User count', sizing_mode='stretch_width', start=10, value=10)
            [3] Param(Input, parameters=['run_sim'], show_name=False, sizing_mode='stretch_width', widgets={'run_sim': {'button_type'...})
        [1] Column(sizing_mode='stretch_width')
            [0] Markdown(str, sizing_mode='stretch_width')
            [1] FloatSlider(name='Bond control variable', sizing_mode='stretch_width')
            [2] IntSlider(end=100, name='Vesting term', sizing_mode='stretch_width', value=5)
            [3] FloatSlider(end=10, name='Min price', sizing_mode='stretch_width', value=0.8)
            [4] IntSlider(end=100000, name='Max payout', sizing_mode='stretch_width', value=10000)
            [5] FloatSlider(end=100, name='Fee', sizing_mode='stretch_width', value=100)
            [6] FloatSlider(end=10, name='Max debt', sizing_mode='stretch_width', step=0.01, value=0.6)
        [2] Column(sizing_mode='stretch_width')
            [0] Markdown(str, sizing_mode='stretch_width')
            [1] Checkbox(name='Add', sizing_mode='stretch_width', value=True)
            [2] FloatSlider(name='Rate', sizing_mode='stretch_width', step=0.01, value=0.02)
            [3] FloatSlider(end=5, name='Target', sizing_mode='stretch_width', step=0.01, value=0.2)
            [4] IntSlider(end=10, name='Buffer', sizing_mode='stretch_width', value=2)
            [5] FloatInput(name='LastBlock', sizing_mode='stretch_width')
    [1] Column(sizing_mode='stretch_width')
        [0] Plotly(Figure, sizing_mode='stretch_width')
        [1] Plotly(Figure, config={'responsive': True}, sizing_mode='stretch_width')
        [2] Plotly(Figure, config={'responsive': True}, sizing_mode='stretch_width')
        [3] Plotly(Figure, config={'responsive': True}, sizing_mode='stretch_width')
        [4] DataFrame(DataFrame, sizing_mode='stretch_width')
        [5] DataFrame(DataFrame, sizing_mode='stretch_width')